In [2]:
%pylab inline

from sklearn.linear_model import *
from sklearn import cross_validation
from scipy import linalg, stats

plt.rcParams['figure.figsize'] = (1 * 10.0, 6.0)
fontsize = 35
plt.rcParams['font.size'] = 30


Populating the interactive namespace from numpy and matplotlib


In [3]:
def test_decoding(img_1, labels_1, img_2, labels_2, n_perm=10000):
    # load the data
    X1 = np.load('data/' + img_1 + '.npy')
    y1 = np.load('data/' + labels_1 + '.npy')
    X2 = np.load('data/' + img_2 + '.npy')
    y2 = np.load('data/' + labels_2 + '.npy')
        
    # number of subjects
    n_sub = y1.shape[0]
  #  print (n_sub)
  #  print (y1.shape[1])
    # initialization
    score = 0.
    scores_perm = []

    # create permutations beforehand so they are shared
    # across subjects
    perms = [np.random.permutation(y1[0].size) for _ in range(n_perm)]
    
    #across maps
 #   perms = [np.random.permutation(y1[1].size) for _ in range(n_perm)]
    # nested cross-validation scheme: leave one run out
    runs = [0] * 8 + [1] * 8 + [2] * 8 + [3] * 8
    cv = cross_validation.LeaveOneLabelOut(runs)

    for i in range(n_sub):
        clf = LogisticRegressionCV(cv=cv)
        clf.fit(X1[i], y1[i].ravel())
        score += clf.score(X2[i], y2[i].ravel())

        # compute the permuted test statistic
        scores_perm_sub = []
        for pi in perms:
            tmp = clf.score(X2[i], y2[i].ravel()[pi])
            scores_perm_sub.append(tmp)
        scores_perm.append(scores_perm_sub)

    # add across subjects
    scores_perm = np.array(scores_perm).sum(0)

    pval = 1 - (score >= scores_perm).mean()
    return pval


In [16]:
# test statistic
def test_stat(A1, A2, B1, B2):
    tmp = np.corrcoef(A1, B1)[0, 1] + \
           np.corrcoef(A2, B2)[0, 1] - \
           np.corrcoef(A1, B2)[0, 1] - \
           np.corrcoef(A2, B1)[0, 1]
    return tmp * tmp

def test_permutation(img_1, labels_1, img_2, labels_2, n_perm=10000):
    # load the data
    X1 = np.load('data/' + img_1 + '.npy', encoding='latin1')
    y1 = np.load('data/' + labels_1 + '.npy', encoding='latin1')
    X2 = np.load('data/' + img_2 + '.npy', encoding='latin1')
    y2 = np.load('data/' + labels_2 + '.npy', encoding='latin1')

    # number of subjects
    n_sub = y1.shape[0]
    # number of runs
    n_runs = 4

    # compute test statistic
    T0 = 0.0
    for i in range(n_sub):
        # indexes of class 1
        idx1 = (y1[i].ravel() == 0)
        # indexes of class 2
        idx2 = (y2[i].ravel() == 1)

        # average across runs
        A1 = np.array(X1[i])[idx1].mean(0)
        A2 = np.array(X1[i])[idx2].mean(0)
        B1 = np.array(X2[i])[idx1].mean(0)
        B2 = np.array(X2[i])[idx2].mean(0)
        T0 += test_stat(A1, A2, B1, B2)


    # compute the permuted statistic
    T_perm = []
    for perm_count in range(n_perm):
        T_subj = 0.0
        
        # permute the non-averaged betas
        # this way we can form more permutations
        perm = np.random.permutation(8 * 4)
    #    perm2 = np.random.permutation(40 * 4)
        for i in range(n_sub):
            # indexes of class 1
            idx1 = (y1[i].ravel() == 0)[perm]
            # indexes of class 2
            idx2 = (y2[i].ravel() == 1)[perm]
            
           
            # average across runs
            A1 = np.array(X1[i])[idx1].mean(0)
            A2 = np.array(X1[i])[idx2].mean(0)
            B1 = np.array(X2[i])[idx1].mean(0)
            B2 = np.array(X2[i])[idx2].mean(0)
            T_subj += test_stat(A1, A2, B1, B2)
        T_perm.append(T_subj)

    pval = 1 - (T0 >= np.array(T_perm)).mean()
    return pval


In [18]:
# compute the decoding p-value for P vs I using a t-test
print('Perc-l+r vs Im-l+r')
# compute score
pval = test_decoding('Perc_l+r_OSC_betamaps', 'Perc_labels', 'Im_l+r_OSC_betamaps', 'Im_labels')
print('p-value decoding: ', pval)

pval = test_permutation('Im_l+r_OSC_betamaps', 'Im_labels', 'Perc_l+r_OSC_betamaps', 'Perc_labels')
print('p-value permutation: ', pval)

Perc-l+r vs Im-l+r
9
4
p-value decoding:  0.0913
p-value permutation:  0.0015


In [6]:
print('Perc-l+r vs Im-l+r (BOLD)')
# same thing using BOLD
pval = test_decoding('Perc_l+r_OSC_BOLD', 'Perc_labels', 'Im_l+r_OSC_BOLD', 'Im_labels')
print('p-value decoding: ', pval)

pval = test_permutation('Perc_l+r_OSC_BOLD', 'Perc_labels', 'Im_l+r_OSC_BOLD', 'Im_labels')
print('p-value permutation: ', pval)


Perc-l+r vs Im-l+r (BOLD)
p-value decoding:  0.6491
p-value permutation:  0.9997


In [19]:
for delay in [0, 1, 2, 3, 4]:
    print("Delay %s"% delay)
    print('VS-l+r vs Perc-l+r (betamaps)')
    pval = test_decoding('VS_delay_%s_l+r_OSC_betamaps' % delay, 'VS_delay_%s_labels' % delay, 
                         'Perc_l+r_OSC_betamaps', 'Im_labels')
    print('p-value decoding: ', pval)

    pval = test_permutation('VS_delay_%s_l+r_OSC_betamaps' % delay, 'VS_delay_%s_labels' % delay, 
                         'Perc_l+r_OSC_betamaps', 'Perc_labels')

    print('p-value permutation: ', pval)
    print()


Delay 0
VS-l+r vs Perc-l+r (betamaps)
9
4
p-value decoding:  0.5637
p-value permutation:  0.0033

Delay 1
VS-l+r vs Perc-l+r (betamaps)
9
4
p-value decoding:  0.0582
p-value permutation:  0.0044

Delay 2
VS-l+r vs Perc-l+r (betamaps)
9
4
p-value decoding:  0.1266
p-value permutation:  0.0

Delay 3
VS-l+r vs Perc-l+r (betamaps)
9
4
p-value decoding:  0.9986
p-value permutation:  0.0

Delay 4
VS-l+r vs Perc-l+r (betamaps)
9
4
p-value decoding:  0.0202
p-value permutation:  0.0053



In [21]:
for delay in [0, 1, 2, 3, 4]:
    print("Delay %s"% delay)
    print('VS-l+r vs Im-l+r (betamaps)')
    pval = test_decoding('VS_delay_%s_l+r_OSC_betamaps' % delay, 'VS_delay_%s_labels' % delay, 
                         'Im_l+r_OSC_betamaps', 'Im_labels')
    print('p-value decoding: ', pval)

    pval = test_permutation('VS_delay_%s_l+r_OSC_betamaps' % delay, 'VS_delay_%s_labels' %delay, 
                         'Im_l+r_OSC_betamaps', 'Im_labels')

    print('p-value permutation: ', pval)
    print()

Delay 0
VS-l+r vs Im-l+r (betamaps)
9
4
p-value decoding:  0.2693
p-value permutation:  0.857

Delay 1
VS-l+r vs Im-l+r (betamaps)
9
4
p-value decoding:  0.3852
p-value permutation:  0.4746

Delay 2
VS-l+r vs Im-l+r (betamaps)
9
4
p-value decoding:  0.1685
p-value permutation:  0.1343

Delay 3
VS-l+r vs Im-l+r (betamaps)
9
4
p-value decoding:  0.6482
p-value permutation:  0.0004

Delay 4
VS-l+r vs Im-l+r (betamaps)
9
4
p-value decoding:  0.2578
p-value permutation:  0.6034

